![Data Quality Banner](Images/banner_data_quality.png)


**Projeto:** Otmização da Dosagem de Reagentes na Flotação.\
**Localização:** Municípios de Craíbas e Arapiraca, Estado de Alagoas.\
**Metodologia:** CRISP-DM.\
**Fase da Metodologia:** Deployment\
**Entregável:** Documentação do Pipeline MLOPS

**Índice**<a id='toc0_'></a>    
1. [Primeiros Passos](#section_1)
    - 1.1 [Estrutura do projeto](#section_11)
    - 1.2 [Elementos do framework de MLOps Kedro](#section_12)
        - [Nodes](#nodes)
        - [Pipelines](#pipelines)
        - [Data Catalog](#data_catalog)
    - 1.3 [Set Up do projeto](#section_13)
        - [Criação de um ambiente virtual via conda](#section_14)
        - [Instalação das dependências do projeto](#section_15)
2. [Pipelines](#section_2)
    - 2.1 [check_raw_data](#section_21)
    - 2.2 [data_processing](#section_22)
    - 2.3 [generate_models_input](#section_23)
    - 2.4 [models_optimization](#section_24)
    - 2.5 [data_science](#section_25)
    - 2.5 [optimization](#section_26)
3. [Fluxo do Projeto](#section_3)
    - 3.1 [Processamento de novos dados](#section_31)
    - 3.2 [Geração da Base Analítica do Modelo Preditivo](#section_32)
    - 3.3 [Verificação da variável que captura mudanças na dinâmica do processo](#section_33)
    - 3.4 [Otimização dos hyperparametros dos modelos preditivos](#section_34)
    - 3.5 [Calibração dos modelos preditivos](#section_35)    
    - 3.6 [Análise de erro/performance](#section_36)
    - 3.7 [Simulação do Otimizador com o Modelo Preditivo](#section_37)
    - 3.8 [Atualização do simulador MVP](#section_38)
4. [Fluxo de Uso](#section_4)
    - 4.1 [Processar Dados Novos](#section_41)
    - 4.2 [Calibrar Modelo Baseline e Executar Simulação de Otimização](#section_42)
    - 4.3 [Otimização de Hiperparâmetros do Modelo Campeão](#section_43)
    - 4.4 [Calibrar Modelo Campeão e Executar Simulação de Otimização](#section_44)
    - 4.5 [Comparar Resultados](#section_45)
    - 4.6 [Atualizar Modelo no MVP (WebApp)](#section_46)

## <a id='section_1'></a>[1. Primeiros Passos](#toc0_)  [&#8593;](#toc0_)

Esta seção descreve a estrutura do projeto e como realizar sua configuração:

- 1.1 [Estrutura do projeto](#section_11)
- 1.2 [Elementos do framework de MLOps Kedro](#section_12)
    - [Nodes](#nodes)
    - [Pipelines](#pipelines)
    - [Data Catalog](#data_catalog)
- 1.3 [Set Up do projeto](#section_13)
    - [Criação de um ambiente virtual via conda](#section_14)
    - [Instalação das dependências do projeto](#section_15)

#### 1.1 <a id='section_11'></a>[ Estrutura do Projeto](#section_1)

Esta seção descreve a estrutura de pastas e arquivos adotada no projeto. Este seguiu o padrão proposto pelo [framework de MLOps Kedro](https://kedro.readthedocs.io/en/stable/) com alguns arquivos adicionais ligados ao *web app MVP de Otimização da Flotação*. Essa organização visa proporcionar uma fácil manutenção, escalabilidade e colaboração durante o desenvolvimento.
Recomenda-se a familiarização com esta estrutura para otimizar a eficiência durante a rotina de utilização, manutenção e evolução das componentes do projeto.

<img src="Images/estrutura_de_pastas.png" alt="Arquivos" width="500"/>


##### **conf**

A pasta `conf` contém todos os arquivos relacionados à configuração do projeto. Isso inclui o [Data Catalog](#data_catalog) e parâmetros de execução. Os arquivos presentes nesta pasta desempenham um papel fundamental na definição e personalização de aspectos operacionais do projeto.

Exemplos de arquivos nesta pasta:

- [`catalog.yml`](#data_catalog) : Descreve os datasets disponíveis e suas fontes.
- `parameters.yml`: Armazena parâmetros de execução, facilitando a flexibilidade e reusabilidade do código.

##### **data**

A pasta `data` é dedicada ao armazenamento de datasets utilizados no projeto. Diferentes tipos de dados, como arquivos `.xlsx`, `.parquet` e `.json`, podem ser encontrados aqui.

Estrutura de exemplo:

```plaintext
data/
|-- raw/
|   |-- dataset_raw.xlsx
|-- processed/
|   |-- dataset_processed.parquet
|-- external/
|   |-- external_data.json
```
##### **src**

A pasta `src` contém todo o código fonte do projeto. As subpastas organizam o código de acordo com suas funcionalidades, tornando mais fácil a navegação e manutenção do código. Nela se encontram os códigos relativos aos [*pipelines*](#pipelines) e seus [*nodes*](#nodes). A descrição destes conceitos será realizado em uma outra seção deste documento (Ver [seção 1.2](#secao_12)).

Exemplos de subpastas:

- `pipelines/`: Contém scripts relacionados a pipelines de tratamento e modelagem preditiva.

##### **docs**

A pasta `docs` é destinada à documentação do projeto. Aqui, você encontrará informações detalhadas sobre o pipeline MLOps, instruções de uso e qualquer outra documentação relevante. O presente documento se encontra na pasta `/docs/guide`.

##### **notebooks**

A pasta `notebooks` abriga todos os Jupyter Notebooks utilizados durante o desenvolvimento do projeto. Esses notebooks podem incluir análises exploratórias, avaliação de resultados de modelos e outros experimentos. Servem como suporte à visualização e análise.

##### **app**

A pasta `app` contém o código fonte do *web app MVP de otimização da flotação* associado ao projeto. Esta pasta é específica ao projeto `mvvflotacao`, não compondo a estrutura padrão do [Kedro](#section_12).

#### 1.2 <a id='section_12'></a>[Elementos do framework de MLOps Kedro](#section_1)



[Kedro](https://kedro.readthedocs.io/en/stable/) é um framework Python de código aberto hospedada pela Linux Foundation (LF AI & Data). Kedro usa as melhores práticas de engenharia de software para ajudar equipes de Cientistas de Dados a construir códigos prontos para produção, diminuindo o esforço usual de refatoração de códigos experimentais.

A utilização do framework busca um desenvolvimento de código modular mantendo-o organizado e seguindo boas práticas a medida que o projeto avança.

Basicamente, os 3 principais [conceitos do Kedro](https://docs.kedro.org/en/stable/get_started/kedro_concepts.html#) são:
- [Nodes](#nodes)
- [Pipelines](#pipelines)
- [Data Catalog](#data_catalog)

Abordaremos cada um dos conceitos nesta seção. Mas recomendamos a [criação de um projeto básico a partir da documentação oficial do Kedro para um treinamento *hands-on*](https://docs.kedro.org/en/stable/tutorial/tutorial_template.html).

Abaixo, observa-se um diagrama que ilustra a estrutura de um projeto kedro padrão (à esquerda) e uma simplificação da estrutura do projeto `mvvflotacao` desenvolvido neste projeto.

<img src="Images/conceito_kedro_comparacao.png" alt="Arquivos" width="1200"/>


##### <a id='nodes'></a> [Nodes](#section_1)
Um *node* (nó) no Kedro é um "empacotador"(*wrapper*) para uma função Python que nomeia as entradas e saídas dessa função. Os *nodes* são os blocos de construção de um *pipeline*, ou seja, um *pipeline* é um conjunto de *nodes*, que por sua vez representam funções em *python*.  Aqui está um exemplo simples de um *node* encontrado no *pipeline* `data_processing`, do projeto `mvvflotacao`:

<img src="Images/node.png" >

O *node* acima empacota/declara a função `preprocess_reagentes` que receberá como *input* os objetos `reagentes_raw` (dataset de dados de dosagem de reagentes) e `params:raw_data_version` (parâmetro de versionamento de dados), ambos declarados no [*Data Catalog*](#data_catalog) do projeto. Como *output* a execução do *node* retornará o dataset `reagentes_pre` (dataset de dosagem de reagentes com tratamentos objetivando a modelagem preditiva). Por fim, todo *node* possui um nome único declarado pelo parâmetro *name*, neste caso, `reagentes_node`.


##### <a id='pipelines'></a> [Pipelines](#section_1)

Um *pipeline* organiza as dependências e a ordem de execução de uma coleção de *nodes*. O *pipeline* determina a ordem de execução dos *nodes* resolvendo dependências e não necessariamente executa os *nodes* na ordem em que são passados. Como exemplo, segue abaixo uma parte do arquivo `pipeline.py` do *pipeline* `data_processing`:

<img src="Images/codigo_pipeline.png" alt="Arquivos" width="500"/>

Observa-se no exemplo acima, a declaração de um conjunto de *nodes* (funções com seus respectivos objetos/arquivos de *input* e *output*) a serem executados no *pipeline* `data_processing`.

Cada *pipeline* do projeto possui dois arquivos principais, o `node.py` que possui as funções escritas em python, e o arquivo `pipeline.py` que organiza os nodes conforme o exemplo anterior.

<img src="Images/estrutura_pipeline.png" alt="Arquivos" width="150"/>

Os *nodes* são declarados no arquivo `pipeline.py` dentro de cada *pipeline* do projeto, referenciando funções escritas, em geral, nos arquivos `nodes.py`. 

Em casos onde a mesma função python é utilizada em mais de um *pipeline*, a mesma pode ser escrita fora do *pipeline* e importada diretamente no arquivo `pipeline.py`. Esta estratégia visa maior consistência geral do projeto. No projeto `mvvflotacao`, as funções compartilhadas estão localizadas em `src/mvvflotacao/shared_code`.



Para criar um novo pipeline seguindo o template mencionado, pode-se utilizar a seguinte linha de código:

```bash
kedro pipeline create data_processing
```

Para executar um *pipeline* específico, deve-se utilizar o seguinte comando:

```bash
kedro run --pipeline="pipeline_name"

#exemplo para o pipeline data_processing
kedro run --pipeline=data_processing
```


Para executar um *pipeline* específico, deve-se utilizar o comando abaixo. Porém lembre-se de verificar se todas as dependências (arquivos de *input* dos *nodes* do *pipeline a ser executado) já estão disponíveis.

```bash
kedro run --pipeline="pipeline_name"

#exemplo para o pipeline data_processing
kedro run --pipeline=data_processing
```

##### <a id='data_catalog'></a> [Data Catalog](#section_1)

O *Data Catalog* do kedro é um registro de todas as fontes de dados utilizadas no projeto. Nele são registrados todos os arquivos de origem (dados brutos), assim como datasets e objetos intermediários criados ao longo de todo fluxo de processamento. O *Data Catalog* do kedro possui diversas funções *built-in* para leitura de arquivos que podem ser utilizados para ler arquivos *.xlsx* e *.csv*, por exemplo. Além disso, neste arquivo devem ser registrados todos os arquivos que serão utilizados como *input* e *output* dos *nodes* do projeto, sejam eles datasets (.parquet), modelos serializados (.pkl), hyperparametros (.json) ou qualquer outro formato utilizado.

Desta forma, os arquivos de *nodes* especificados nos *pipelines* utilizarão como referência apenas os nomes dos arquivos especificados no *Data Catalog*, previamente configurados, não necessitando especificar seu *path* de forma explícita no código.

Segue abaixo um trecho do arquivo *Data Catalog* do projeto `mvvflotacao` encontrado em `mvvflotacao/conf/base/catalog.yml`:

<img src="Images/catalog.png" alt="Arquivos" width="400"/> 

Observa-se no exemplo acima a especificação dos datasets `reagentes_raw` e `blend_raw` que estão armazedos no formato *.xlsx (ExcelDataset)*. Estes representam *dados brutos* recebidos diretamente dos bancos de dados da planta de flotação da MVV. Neste arquivo, encontra-se também a especificação dos datasets `cata_controle_pims_pre` e `balanco_de_massas_pre` armazeados em formato *.pq (ParquetDataset)*, um formato comumente utilizado em datasets intermediários de fluxos MLOps devido à sua velocidade de leitura e escrita.

Observe agora o exemplo do *node* `reagentes_node`, encontrado em `./src/mvvflotacao/piepines/data_processing/pipeline.py`, responsável pelo pré-processamento inicial do dataset `reagentes_raw` (ligado à dosagem de reagentes). No exemplo, apenas os *inputs* e *outputs* são definidos seguindo os nomes registrados no *Data Catalog*. Com isso, não há a necessidade de especificar parâmetros de leitura e escritas adicionais.

<img src="Images/node_example_data_catalog.png" alt="Arquivos" width="400"/> 



#### 1.3 <a id='section_13'></a>[ Set Up do projeto](#section_1)

Para garantir a consistência do ambiente de desenvolvimento, recomenda-se a criação de um ambiente virtual específico para o projeto. Esta seção aborda o passo-a-passo encontrado na [Documentação Oficial do Kedro](https://docs.kedro.org/en/stable/get_started/install.html).

##### **Instalação do Conda**

Se você ainda não utiliza o Conda como gerenciador de ambientes virtuais, recomenda-se instalá-lo seguindo as instruções em [Conda Installation](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

#### <a id='section_14'></a>[Criação de um Novo Ambiente Virtual com Conda](#section_1)

```bash
# Substitua `my_kedro_env` pelo nome que deseja dar ao ambiente virtual
conda create --name my_kedro_env python=3.10

#Ative o ambiente virtual
conda activate my_kedro_env
```


Ao executar esses comandos, você terá criado um ambiente virtual isolado com o nome my_kedro_env e o Python 3.10 instalado. Certifique-se de ativar o ambiente virtual sempre que estiver trabalhando no projeto

#### <a id='section_15'></a>[Instalação das Dependências do Projeto](#section_1)

Certifique-se de estar na raiz do projeto antes de executar os comandos a seguir.

```bash
#Instale as dependências do projeto a partir do arquivo requirements.txt
pip install -r src/requirements.txt

# Ou inicialize pelo comando do Kedro
kedro install 
```


Ao seguir esses passos, você terá configurado um ambiente virtual com Conda e instalado as dependências do projeto, conforme as melhores práticas do Kedro. Certifique-se de manter o arquivo requirements.txt atualizado à medida que novas dependências forem adicionadas ao projeto. Isso garantirá consistência entre ambientes de desenvolvimento.

## 2. <a id='section_2'></a>[Pipelines do Projeto `mvvflotacao`](#toc0_) [&#8593;](#toc0_)


O projeto `mvvflotacao` possui um conjunto de 6 *pipelines* principais encontrados em `./src/mvvflotacao/pipelines`, conforme a imagem abaixo:

<img src="Images/pipelines_projeto_folders.png" alt="Arquivos" width="200"/>

Os *pipelines* compõem um grande fluxo que percorre as etapas de um projeto de *Advanced Analytics*, realizando desde à leitura de *dados brutos* recebidos diretamente das fontes oficias da MVV até a análise de otimização da dosagem de reagentes a partir do modelo preditivo treinado. 

Nesta seção realizaremos um resumo dos objetivos e fuções que compõem cada um dos  6 *pipelines* do projeto:

- 2.1 [check_raw_data](#section_21)
- 2.2 [data_processing](#section_22)
- 2.3 [generate_models_input](#section_23)
- 2.4 [models_optimization](#section_24)
- 2.5 [data_science](#section_25)
- 2.5 [optimization](#section_26)

<img src="Images/pipelines_detalhados.png" alt="Arquivos" width="700"/>


#### 2.1 <a id='section_21'></a> [ check_raw_data](#section_2)

Este pipeline apresenta funções destinadas à verificar e comparar conjuntos de dados novos com conjuntos dados anteriores e verificar a existência de inconsistências.. Abaixo está um resumo das principais funcionalidades e lógicas presentes no código:

##### Funções de Verificação

`_check_column_names`:
Compara os nomes de colunas entre dois dataframes e registra qualquer diferença encontrada.

`_check_column_type`:
Compara os tipos de dados das colunas entre dois dataframes e registra qualquer diferença encontrada.

`_check_hist_values`:
Compara as variações históricas nos dados entre dois dataframes e registra divergências.

##### Função Principal `check_rawdata`

- Recebe dois conjuntos de dados brutos, uma nova versão e uma versão anterior, e um nome de dataset.
- Carrega os dataframes correspondentes às versões fornecidas.
- Para o dataset 'reagentes', aplica o tratamento específico.
- Realiza uma série de verificações utilizando as funções previamente definidas (`_check_column_names`, `_check_column_types`, `_check_hist_values`).
- Registra mensagens informativas e de aviso conforme as diferenças ou divergências encontradas.

O código visa garantir a consistência e qualidade dos dados ao longo do tempo, fornecendo informações detalhadas sobre qualquer alteração nos conjuntos de dados brutos. As verificações abrangem aspectos como nomes e tipos de colunas, bem como variações históricas, contribuindo para a manutenção da integridade dos dados no contexto do recebimento de novos dados.

##### Utilização do Pipeline `check_raw_data`

Para utilizar o pipeline `check_raw_data`, é necessário:

-   Estrutura na nuvem ou localmente (referenciados no catálogo) com pastas contendo o nome respectivo ao nome da base, e arquivos referentes às datas (Os parâmetros do `check_raw_data` devem ser referentes ao nome dos arquivos de datas).
-   Executar o pipeline com o comando `kedro run --pipeline=check_raw_data`.
-   Garantir que os requisitos dos arquivos estruturados estejam atendidos.

##### Análise dos Outputs

Após a execução do pipeline, é importante analisar os outputs no terminal. Isso inclui verificar se as variáveis importantes das tabelas tiveram mudanças significativas, como alterações de nome das colunas, mudanças nos tipos de dados ou variações históricas inesperadas. Essas informações são cruciais para entender a integridade e a consistência dos dados recebidos.

<img src="Images/exemplo_check_raw_data.png" alt="Arquivos" width="700"/>

- Na análise realizada, não foram identificadas alterações nos nomes e tipos das colunas. No entanto, foram observadas sutis variações nas datas nos últimos registros da base designada como "previous_raw_data".

#### 2.2  <a id='section_22'></a> [data_processing](#section_2)

O código apresenta funções dedicadas ao pré-processamento de dados brutos, recebidos diretamente dos sistemas da planta de flotação da MVV. O *pipeline* visa ajustar o formato dos arquivos recebidos, ajustando estruturas como *data types*, formatos de data. Além disso, o *pipeline* realiza a preparação adequada dos dados ao treinamento de modelos preditivos, como a remoção de períodos inconsistentes e a agregações temporal. Abaixo está um resumo das principais funcionalidades e lógicas presentes no código:

#####  **Função `preprocess_rawdata`**
   - Converte colunas de tipo 'object' para 'string' em um DataFrame.
   - Aceita um DataFrame ou um dicionário de funções para carregar DataFrames a partir de diferentes versões de dados brutos.

##### **Funções de Pré-processamento Específicas para Dados de 'reagentes', 'laboratorio' e 'blend'**
   - **`preprocess_reagentes`**
      - Carrega e pré-processa dados específicos do dataset 'reagentes'.
      - Realiza renomeação de colunas, manipulação de datas e horas, filtragem e preenchimento de valores faltantes.
   - **`preprocess_laboratorio`**
      - Carrega e pré-processa dados específicos do dataset 'laboratorio'.
      - Realiza filtragem, manipulação de datas e horas, e cria cópias com ajustes temporais.
   - **`preprocess_blend`**
      - Carrega e pré-processa dados específicos do dataset 'blend'.
      - Realiza combinação de colunas de data e hora, filtragem e seleção de colunas relevantes.

#####  **Função `pivoting_blend`**
   - Realiza a pivotação e pré-processamento dos dados de 'blend' para calcular a porcentagem de 'Tipo_Material' por hora.

##### **Funções de Manipulação de Dados**
   - **`filter_columns_before_merge`**
      - Filtra colunas de um DataFrame baseado em uma tabela de metadados fornecida.
   - **`merge_raw_data`**
      - Mescla vários DataFrames usando a coluna 'DATA' como chave.
   - **`remove_index_before_and_after`**
      - Remove índices específicos e 'n' índices antes e depois de cada um.
   - **`filter_data`**
      -   Filtra um DataFrame removendo períodos específicos inconsistentes.
   - **`agg_by_3h`:**
      -   Agrega dados em intervalos de 3 horas, aplicando medidas estatísticas definidas nos parâmetros da pipeline.
   - **`add_new_features`:**
      -   Faz a média de variáveis similares para evitar problemas de colinearidade.
      
As funções do código visam garantir a consistência, tratamento e agregação apropriados dos dados brutos, fornecendo um conjunto de dados processado e pronto para análise preditiva.

##### **Função de Divisão dos Dados**
   - **`split_data`:**
      -   Função para divisão dos dados em conjuntos de treinamento e teste, realiza essa divisão de maneira aleatória e também ordenando pela data.


##### Execução do Pipeline `data_processing`

Para executar este pipeline, utilize o comando:

`kedro run --pipeline=data_processing`

Este comando iniciará o processo de pré-processamento dos dados, aplicando todas as funções e lógicas descritas acima. É importante garantir que os dados brutos estejam corretamente formatados e disponíveis para que o pipeline funcione conforme esperado.

#### 2.3 <a id='section_23'></a> [ generate_models_inputs](#section_2)

Etapa de tratamento de dados prévia à utilização em técnicas de Machine Learning. Realiza tratamentos como feature engineering e remoção da autocorrelação nos dados. O objetivo deste *pipeline* é gerar os datasets utilizados como *input* direto no *pipeline* de treinamento dos modelos preditivos.

##### **Limpeza de Dados e Registro de Avisos**

- O código inclui seções dedicadas à limpeza de dados, como a remoção de colunas com uma porcentagem alta de valores nulos.

#####  ***Feature Selection* e Pré-processamento de Dados**

- **`select_features_from_step_columns`**: Seleciona colunas de recursos relevantes, considerando exclusões específicas e incluindo a coluna alvo.
- **`get_target_name_for_model`**: Recupera o nome da coluna alvo para um modelo específico.
- **`correct_type`**: Realiza a formatação adequada do DataFrame, definindo o índice, removendo colunas de data e convertendo tipos de colunas.
- **`remove_autocorrelation`**: Remove a autocorrelação dos dados usando uma abordagem de amostragem sistemática.

##### ***Feature Engineering***

- **`create_lag_features`**: Gera características de defasagem para as colunas do DataFrame.
- **`create_difflag_feature`**: Cria um novo DataFrame com características de diferença com base em colunas defasadas.
- **`remove_outliers`**: Remove outliers dos conjuntos de treinamento e teste com base em determinadas colunas.

##### **Parametrização de Medidas no Feature Engineering**

-   No processo de *feature engineering*, as medidas são parametrizadas e podem ser selecionadas entre opções como mínimo (`min`), máximo (`max`), mediana (`median`), diferença entre mínimo e máximo (`diff_min_max`), e mediana (`median`). Isso permite uma flexibilidade na criação de features, adaptando-se às necessidades específicas de cada modelo.

##### **Geração de Conjunto de Dados para Modelos Específicos**

- Funções como `generate_dataset_for_conc_cd`, `generate_dataset_for_rej_rougher`, `generate_dataset_for_rec_global`, etc., geram conjuntos de dados específicos para modelos específicos com base em etapas e metadados fornecidos.

Para executar este pipeline, utilize o comando:

```bash
kedro run --pipeline=generate_models_inputs
```

Este comando inicia o processo de geração de inputs para os modelos, aplicando todas as técnicas de pré-processamento, feature engineering e divisão de dados conforme descrito acima. É importante garantir que os parâmetros e os dados de entrada estejam corretamente configurados para que o pipeline funcione de maneira eficaz.

#### 2.4 <a id='section_24'></a> [ models_optimization](#section_2)

Este pipeline é projetado para ajustar hiperparâmetros do modelo EBM e encontrar a combinação que resulta em menor RMSE médio a dentre amostras sistemáticas dos dados. 

As amostragens sistemáticas são realizadas em pipelines anteriores, onde em resumo, extrai-se 3 amostras dos dados completos intercalando sequencialmente às observações históricas às amostras. Exemplificando, o exemplo 1 (setup da flotação em um horário específico) nos dados de treino pertencerá à amostra 1, o exemplo 2 pertencerá à amostra 2, o exemplo 3 à amostra 3, o exemplo 4 percenterá à amostra 1, o exemplo 5 à amostra 2, assim por diante. Desta forma, geram-se 3 amostras similares para verificação da consistência dos aprendizados resultantes do treinamento de modelos preditivos.

No processo de busca dos melhores hiperparâmetros para os modelos preditivos, neste caso, o `ExplainableBoostingRegressor` da biblioteca `interpret`, são performadas as etapas abaixo:

##### **Otimização de Hiperparâmetros (`optimize_ebm_model`):**
- Define um espaço de busca de hiperparâmetros para a otimização usando a biblioteca `hyperopt`.
- Divide os dados em treinamento e teste para cada uma das três amostras sistemáticas.
- Define uma função objetivo para a otimização que retorna a média dos RMSEs nas três amostras.
- Realiza a otimização usando a estratégia de otimização Tree-structured Parzen Estimator (`tpe.suggest`).
- Executa a otimização em iterações, salvando parcialmente os resultados a cada iteração.
- O número de iterações é determinado pelos parâmetros do modelo (`params['max_iter']` e `params['save_iter']`).
- Salva os resultados da otimização em um arquivo pickle para evitar recomeçar a otimização em caso de interrupção.
- O resultado salvo é utilizado no *pipeline* de treinamento do modelo preditivo [`data_science`](#section_25).

##### **Funções Auxiliares**

**`split_data`**:
- A função recebe um DataFrame completo (`full_data`) e o divide em conjuntos de treinamento e teste.
- A divisão é feita ordenando os dados pela coluna 'DATA' e pegando os primeiros 80% para treinamento e o restante para teste.

**`result_model`**:
- Treina um modelo EBM nos dados de treinamento usando hiperparâmetros fornecidos.
- Avalia o modelo treinado nos dados de teste usando a função `calc_rmse`.

**`calc_rmse`**:
- Calcula o Root Mean Squared Error (RMSE) das previsões do modelo nos dados de teste.


##### **Notas Adicionais:**
- O código suporta o carregamento parcial dos resultados da otimização para retomar de onde parou, se necessário.
- Os resultados da otimização são salvos em um arquivo pickle para persistência entre execuções.

Para executar toda a pipeline de `models_optimization`, deve-se rodar o seguinte código no terminal do anaconda:

```bash
kedro run --pipeline=models_optimization
```

Esta instrução, quando executada, iniciará o processo de otimização de modelo descrito na seção, rodando todas as etapas necessárias para encontrar os melhores hiperparâmetros para o modelo EBM.

#### 2.5 <a id='section_25'></a> [data_science](#section_2)

O *pipeline* define funções para treinamento e avaliação dos resultados de modelos preditivos. As funções recebem como *input* os dados tratados nas etapas anteriores, assim como os parâmetros otimizados gerados no *pipeline* `models_optimization`. Como resultado, o *pipeline* retorna uma série de cursos para avaliação de performance, como gráficos e métricas, assim como o modelo treinado serializado.

As principais funções incluem:


##### **Treinamento do modelo preditivo**

**`train_model`:** treina o modelo de machine learning flexível e parametrizado pelo arquivo de configuração. Atualmente, o pipeline suporta modelos do tipo ExplainableBoostingRegressor, uma implementação dos algoritmos de Boosting Machine porém com recursos que auxiliam em sua interpretabilidade. Pode-se treinar apenas um modelo ou um ensemble de EBMs. Novos modelos podem ser desenvolvidos para serem executados pelo pipeline (ver `src/mvvflotacao/shared_code/modeling/README.md`).

##### **Previsão para avaliação dos Resultados**

**`data_test_predict`:** Recebe um conjunto de dados de teste e um modelo treinado, faz previsões nos valores alvo e adiciona uma coluna 'Prediction' ao DataFrame.

**`data_full_predict`:** Prevê valores alvo em um conjunto de dados completo usando um modelo treinado. Também realiza correção de tipo no DataFrame, convertendo colunas 'Float64' para 'float64' e removendo colunas 'datetime64[ns]'.

##### **Métricas e Gráficos para Análise**

**`calculate_metrics`:** Calcula métricas de avaliação (RMSE, MAE, R-squared) para um conjunto de dados de teste fornecido.

**`create_model_snapshot`:** Cria um instantâneo das métricas e parâmetros de um modelo para documentação. Inclui informações como nome do modelo, nomes das features, comentários e vários parâmetros do modelo.

**`consolidate_snapshots`:** Combina dados de várias partições (identificadas por chaves) em um único DataFrame. Utiliza funções fornecidas no dicionário 'partitions' para carregar dados de cada partição.

**`create_valid_frame`:** Gera um DataFrame contendo dados de treinamento e teste, juntamente com previsões de teste. Renomeia a coluna 'predictions' para 'Prediction_Valid'.

##### *Execução*

Para executar todo o pipeline data_science, a instrução de comando é:

```bash
kedro run --pipeline=data_science
```

Este comando iniciará o processo descrito no pipeline data_science, realizando todas as etapas necessárias para treinar e gerar os arquivos para análise dos modelos preditivos.

#### 2.6 <a id='section_26'></a> [ optimization](#section_2)

Executa análises de otimização de variáveis de decisão (ex: dosagem de reagentes e vazão de ar) usando um modelo treinado no *pipeline* `data_science`. Ele opera com base nos algorítimos de otimização gridsearch e otimização bayesiana.

#####  Função de Simulação (`simulate`):
- Gera cenários de simulação com base em uma instância da classe `Scenarios`.
- Para cada cenário, realiza otimização utilizando um otimizador de busca em grade (`GridSearchOptimizer`) e um otimizador bayesiano (`BayesianOptimizer`).
- Armazena os resultados das simulações.

##### Processamento de Resultados (`process_results`):
- Constrói um relatório de resultados usando a classe `SimulationResults` do módulo `optimizer_utils`.
- Gera visualizações, incluindo gráficos de desempenho em relação a diferentes parâmetros.

##### Otimizadores:
- `GridSearchOptimizer`: Otimizador de busca em grade para avaliar todas as combinações possíveis.
- `ItGridSearchOptimizer`: Versão iterativa do otimizador de busca em grade.
- `BayesianOptimizer`: Otimizador bayesiano utilizando a biblioteca Optuna.
- `GAOptimizer`: Otimizador genético (não implementado).

##### *Execução*

Para executar todo o pipeline optimization, a instrução de comando é:

```bash
kedro run --pipeline=optimization
```

- Este comando iniciará o processo de otimização descrito no pipeline optimization, onde serão exploradas diferentes combinações de variáveis de decisão para otimizar o desempenho do modelo.


## 3 <a id='section_3'></a>[Fluxo do Projeto](#toc0_) [&#8593;](#toc0_)


O fluxo do projeto abrange diversas etapas, desde o processamento de novos dados até a atualização do Simulador MVP. Cada fase desempenha um papel crucial no desenvolvimento, calibração e otimização dos modelos preditivos. A seguir, detalhamos cada uma dessas etapas:

  - 3.1 [Processamento de novos dados](#section_31)
  - 3.2 [Geração da Base Analítica do Modelo Preditivo](#section_32)
  - 3.3 [Verificação da variável que captura mudanças na dinâmica do processo](#section_33)
  - 3.4 [Otimização dos hyperparametros dos modelos preditivos](#section_34)
  - 3.5 [Calibração dos modelos preditivos](#section_35)    
  - 3.6 [Análise de erro/performance](#section_36)
  - 3.7 [Simulação do Otimizador com o Modelo Preditivo](#section_37)
  - 3.8 [Atualização do simulador MVP](#section_38)


#### 3.1  <a id='section_31'></a>[Processamento de Novos Dados](#section_3)

Nesta fase, os dados recém-coletados ou atualizados são processados e integrados ao projeto. Isso inclui a validação, limpeza e transformação dos dados para garantir sua qualidade e utilidade nos modelos preditivos.

Para essa etapa são utilizados os pipelines [`check_raw_data`](#section_21) e [`data_processing`](#section_22).

Para atualizar os dados, é necessário alterar os os parâmetros de data no arquivo '\conf\base\parameters.yml'.

```yaml
raw_data_version: '2023_12_18-07_00_00'
previous_raw_data_version: '2023_12_10-17_00_00'
models_to_solve: 
  - conc_cd
  - rec_global
```

- raw_data_version: data/hora da nova base
- previous_raw_data_version: data/hora anterior
- models_to_solve: lista com os problemas que serão resolvidos por todos os pipelines. Neste exemplo, vemos que todas as execuções irão considerar os problemas de teor de cobre no CD e recuperação global. Caso o usuário queira executar os pipelines para somente um dos problemas, é só alterar este parâmetro.

```yaml
split_data:
  test_size: 0.15

measures: ["min", "max", "median", "diff_min_max"]
```

- split_data:
    - test_size: porcentagem referente ao dataset de teste, os demais registros entrarão no dataset de treino 
- measures: medidas para agregar as observações dentro do intervalo de horas, por padrão é utilizada a média mas podem ser adicionadas outras medidas (opções: 'min','max','median','diff_min_max')

**Armazenamento na Cloud**: 

Esse pipeline também suporta o armazenamento dos datasets calculados em buckets no Microsoft Azure Data Storage. Esse tipo de armazenamento facilita o compartilhamento dos outputs desse pipeline entre múltiplas pessoas. Nesse caso, é necessário que somente uma execução do pipeline data_processing seja feita e os resultados serão armazenados no bucket. A partir desse momento, as execuções dos demais pipelines utilizam os dados já pré-processados e salvos no bucket, economizando em tempo de processamento e também padronizando os resultados de execuções em múltiplos computadores.

Para isso, é necessário alterar o catálogo de dados utilizados pelo Kedro em `conf/base/catalog.yml`. Um exemplo de catálogo com armazenamento no Data Storage pode ser visto em `conf/cloud/catalog.yml`. Um exemplo de entrada no arquivo para armezanamento em cloud é apresentado abaixo: 

```yaml
balanco_de_massas_raw:
  type: PartitionedDataset
  path: abfs://mvvflotacao/01_raw/balanco_de_massas/
  credentials: azure_data_storage
  filename_suffix: .xlsx
  dataset:
    type: pandas.ExcelDataSet
    load_args:
      engine: openpyxl
      decimal: "."
      parse_dates: ["DATA"]
```

O atributo `path` deve refletir o endereço/bucket onde o dado está salvo.

*Observação*: para garantir o acesso a bases armazenadas em serviços de nuvem, é necessário fornecer um arquivo de [credenciais](https://docs.kedro.org/en/stable/data/data_catalog.html) ao Kedro. Certifique-se de incluir esse arquivo para que o Kedro possa estabelecer a conexão necessária durante a execução do pipeline.

**Armazenamento Local**:

Para armazenamento local, as bases de dados devem ser adicionadas com o nome correspondente no arquivo 'catalog.yml'.

```yaml
laboratorio_raw:
  type: pandas.ExcelDataSet
  filepath: data/01_raw_data/laboratorio.xlsx
  load_args:
    engine: openpyxl
    decimal: "."
    parse_dates: ["DATA"]
```

Neste caso, o passo inicial para processar os dados seria adicioanr as bases na pasta ` data/01_raw_data/`, com os respectivos nomes no catálogo (nesse caso, laboratorio.xlsx).

**Kedro Env:** 

O projeto é configurado para reconhecer o `catalog.yml` dentro de `conf\base` como a configuração padrão. Se desejar utilizar outro catálogo, basta incluir o argumento `--env` seguido pelo nome da pasta que contém o respectivo `catalog.yml`.

Para exemplificar, ao executar o pipeline completo de `data_processing` com armazenamento em nuvem, utilize o comando `kedro run --pipeline=data_processing --env=cloud`. Neste caso, a referência é feita à subpasta `cloud` dentro de `base`, onde o arquivo `catalog.yml` específico para o armazenamento em nuvem está localizado.

#### 3.2  <a id='section_32'></a>[Geração da Base Analítica do Modelo Preditivo](#section_3)

A etapa de geração das bases analíticas foi criada para determinar os inputs de cada modelo, nessa etapa deverá ser executado o pipeline [`generate_models_inputs`](#section_23). Nesse pipeline serão gerados os datasets utilizados para cada modelo, 

Caso seja necessário alterar as features utilizadas no modelo deve-se alterar o arquivo '\conf\base\parameters_generate_models_inputs.yml'. Nesse arquivo, além de alterar as features, é possível alterar alguns parâmetros como a remoção de outliers do dataset de treino e/ou teste, remoção de valores nulos e zerados, criação de lags e seleção de agregações como mínimo, máximo, etc.

Os parâmetros desta etapa estão detalhados a seguir:
- **target:** nome da variável resposta (target)
- **remove_outliers:** 
    - **train:** True/False, determina se deveram ser removidos os outliers do dataset de treino 
    - **test:** True/False, determina se deveram ser removidos os outliers do dataset de teste 
- **lag:** True/False, parâmetro para criação de variáveis em lag
- **diff_lag:** True/False, parâmetro para criação de variáveis com as diferenças entre os lags 1 e 2
- **filter_val_dardo_cd:** True/False, booleano para filtragem pelo campo de Válvula Dardo
- **dropna:** True/False
- **remove_0_values:** True/False, booleano para remoção das observações que possuem valor zero
- **measures:** medidas para agregar as observações dentro do intervalo de horas, por padrão é utilizada a média mas podem ser adicionadas outras medidas (opções: 'min','max','median','diff_min_max')
- **features:** lista com os nomes de todas as variáveis que serão utilizadas no modelo
- **steps:** lista com todas as etapas que serão utilizadas no modelo
- **dynamic_features:** períodos com mudança de comportamento (explicação detalhada na seção 3.3)

Importante: 

Para gerar as bases pela lista de variáveis, a função do modelo nos nodes do generate_models_inputs deverá chamar a função _generate_dataset_by_params_ e utilizar o parâmetro _features_ conforme exemplo abaixo:

```yaml
generate_dataset_for_conc_cd:
  target: "CONC_ROUG_FC01_CUT"
  remove_outliers:
    train: True
    test: False
  lag: False
  diff_lag: False
  filter_val_dardo_cd: True
  dropna: True
  remove_0_values: True
  measures: ['min','max','median','diff_min_max']
  features:
    [
      'DEPRESSOR_CD_LH',
      'Sulfetado_MG_Estoque',
      'VELOC_BF1_MOINHO',
    ]
  dynamic_feature: 
    [
      '2023-07-03T03:00:00.000000000', 
      '2023-07-11T21:00:00.000000000',
    ]
```

![nodes_generate_models_inputs_features](Images/nodes_generate_models_inputs_features.png)

Para gerar as bases pelas etapas, a função de geração deverá chamar a função _generate_dataset_from_steps_ e utilizar o parâmetro _steps_ conforme exemplo abaixo:

![nodes_generate_models_inputs_features](Images/nodes_generate_models_inputs_features.png)

![parameters_generate_models_inputs_steps](Images/parameters_generate_models_inputs_steps.png)

#### 3.3 <a id='section_33'></a>[Verificação da variável que captura mudanças na dinâmica do processo](#section_3)

A variável de dinâmica do processo é utilizada nos modelos com intuito de trazer informações sobre a constante evolução dos processos produtivos na MVV, seja por mudanças da planta, insumos e/ou operacionais.

Esta feature é calculada analisando a série temporal da variável target escolhida. No exemplo de teor de cobre no CD, tem-se o gráfico abaixo:

<img src="Images/dynamic.png" alt="Otimização" width="1500"/>

Cada linha pontilhada se refere a alterações na dinâmica da variável de teor de cobre, ou seja, o algoritmo de offline change point detection considera que esta classificação minimiza as diferenças de comportamento em cada segmento. Ver o pacote [Ruptures](https://centre-borelli.github.io/ruptures-docs/) para mais informações.

A execução desta análise pode ser feita com o Jupyter Notebook `notebooks/Dynamic Feature.ipynb`. As informações de mudança de dinâmica são passadas aos pipelines de geração de features via arquivo de configuração como uma lista de datas, ou seja, a lista de pontos onde ocorre uma mudança de dinâmica. Ver o arquivo de configuração `conf/base/parameters_generate_models_inputs.yml` para um exemplo.

Caso sejam adquiridos mais dados, é importante re-executar essa análise para verificar se houve alguma mudança de dinâmica recente.

#### 3.4 <a id='section_34'></a>[Otimização dos hyperparametros dos modelos preditivos](#section_3)

Nesta fase, são realizados experimentos para encontrar os melhores hiperparâmetros dos modelos preditivos. Isso envolve a busca por combinações ideais que maximizem o desempenho do modelo. Para otimizar os hiperparâmetros, temos o pipeline [`models_optimization`](#section_24) onde é feita uma otimização bayesiana para encontrar os melhores hiperparâmetros para o modelo.

Nos parâmetos da otimização de hiperparâmetros ('\conf\base\parameters_models_optimization.yml'), temos:

![parameters_models_optimization](Images/parameters_models_optimization.png)

- load_trials: True/False, parâmetro para carregar a última tentativa ou começar uma nova otimização
- max_iter: quantidade de tentativas (iterações) que a otimização vai realizar para encontrar os melhores hiperparâmetros
- save_iter: parâmetro para salvar os resultados a cada x iterações, pois caso haja algum problema no meio da execução temos os resultados parciais

Ao final do processo de otimização, o sistema salva automaticamente os hiperparâmetros que produziram os melhores resultados no arquivo padrão do modelo. Isso assegura que os avanços obtidos na otimização sejam preservados e facilmente acessíveis para uso futuro.

#### 3.5 <a id='section_35'></a>[Calibração dos modelos preditivos](#section_3)

A etapa de calibração do modelo é dedicada ao ajuste fino dos modelos preditivos. Os algoritmos são treinados utilizando conjuntos de dados históricos, refinando seus parâmetros para melhor se adaptarem às características específicas do sistema em questão. Essa etapa foi implementada no pipeline [`data_science`](#section_25).

Parâmetros da calibração do modelo ('\conf\base\parameters_data_science.yml'):

![parameters_data_science](Images/parameters_data_science.png)

- train_ebm_model:
    - latest: True/False, para selecionar o último arquivo de hiperparâmetro salvo
    - hyperparameters_path: caminho com o arquivo de hiperparâmetros que deve ser utilizado caso o parâmetro 'latest' seja falso
- run_conformal_nodes: True/False

- model:
    - type: tipo do modelo que será treinado
    - randomly_dataset_s0: True/False, booleano para treinar o modelo com o dataset dividido aleatoriamente
    - by_date_dataset_s0: True/False, booleano para treinar o modelo com o dataset dividido por data
    - features: lista com as features que serão utilizadas como input no modelo

#### 3.6 <a id='section_36'></a>[Análise de erro/performance](#section_3)

Ao final da execução dos pipelines `generate_models_input` e `data_science`, têm-se os modelos treinados e salvos na pasta `data/06_models`. Neste momento, é possível analisar os erros do modelo executando o Jupyter Notebook `notebooks/Modelagem-TeorCu-Cd new pipeline.ipynb`. A execução deste notebook gera como o relatório de métricas de performance para o modelo treinado. 

O relatório do modelo pode ser visto na figura abaixo. Neste relatório, mostram-se as métricas de performance do modelo, principalmente a Raiz do Erro Quadrático Médio (RMSE, em inglês) e o Coeficiente de Determinação R2 em duas abordagens de treinamento distintas: com split temporal e com split aleatório. 

<img src="Images/newplot.png" alt="Otimização" width="800"/>

No split temporal, utiliza-se o último mês de dados como teste e o restante como treino do modelo. Já no split aleatório, define-se que uma amostragem aleatória de 15% da base como teste e o restante é destinado ao treino. Reforça-se que o modelo destinado a ser utilizado em produção é sempre o com split aleatório, pois utiliza dados mais recentes para ser treinado, porém é interessante analisar o modelo por split temporal para entender as diferentes dinâmicas dos dados e do modelo ao longo do tempo.

Por fim, o relatório acima mostra as 10 principais variáveis identificadas pelos modelos treinados nos diferentes paradigmas. 

**Determinação do Melhor Modelo:**

No processo de escolha e determinação do modelo a ser utilizado em produção, recomenda-se as seguintes ações: 
- Analisar resultados baseline, ou seja, resultados que devem ser superados pelas modelagens mais complexas. Para isso, é necessário a execução do notebook `notebooks/Modelagem-TeorCu-Cd-Baseline.ipynb`. Nesse notebook, resultados como Lag 1 e média simples serão apresentados em termos das métricas de performance descritas anteriormente.
- Gerar este mesmo relatório para modelos mais simples com menor conjunto de variáveis, ou seja, um modelo EBM simples com as variáveis mais importantes do ponto de vista de negócio. Um exemplo deste tipo de modelo mais simples pode ser treinado alterando o arquivo de configuração `conf/base/parameters_data_science.yml` para: 

```yaml
model_conc_cd:
  type: ebm
  randomly_dataset_s0: True
  by_date_dataset_s0: False
  features:
  [
      'FLOT_AL_MASSA',
      'P20_MOAGEM',
      'P99_MOAGEM',
      'PSI_OVER_CICLO',
      'ALIM_FLOT_CU_TOT',
      'ESPUMA_ROUGHER_COND',
      'PH_ROUGHER_COND',
      'VAL_DARDO_CD',
      'VAZAO_AR_ROUGHER_COND',
      'ALIM_FLOT_PER_SOL',
      'ALIM_FLOT_FE',
      'ALIM_FLOT_MG',
      'ALIM_FLOT_NI',
      'Espumante (g/t)_CD',
      'CONC_ROUG_FC01_CUT',
      'Sulfetado_LG',
      'Sulfetado_HG',
      'Sulfetado_MG',
      'Sulfetado_SHG'
  ]
```

- Avaliar os resultados do modelo anteriormente utilizado em produção para os dados mais recentes coletados. Por exemplo, considerando que o modelo utilizado atualmente foi treinado com dados até Novembro/23. Pode-se testar este mesmo modelo para os dados coletados em Dezembro/23. Este teste é comparável com um modelo treinado via split temporal até Novembro/23 e testado até Dezembro/23;
- Comparar os resultados obtidos nos itens anteriores com o relatório extraído do notebook `notebooks/Modelagem-TeorCu-Cd new pipeline.ipynb`.


#### 3.7 <a id='section_37'></a>[Simulação do Otimizador com o Modelo Preditivo](#section_3)

O objetivo deste pipeline é testar o comportamento do modelo frente a otimização das variáveis de decisão escolhidas. 

Primeiramente, pode-se definir o problema de otimização como: $$\underset{V_d}{\text{argmax}} \, \mathcal{M}(V_d, X),$$ 

ou seja, deseja-se encontrar o melhor conjunto de variáveis de decisão $V_d$ que maximiza a resposta do Modelo $\mathcal{M}$, considerando também outras variáveis de contexto $X$. Por exemplo, nos pipelines anteriores, define-se um modelo $\hat{y} = \mathcal{M}(V_d, X)$ que prevê o teor de cobre no CD $\hat{y}$ dado um conjunto de features que é composto por $V_d$ e $X$.

Para se avaliar comportamento do modelo na otimização, executa-se o otimizador para $n$ cenários distintos, ou seja, $n$ configurações diferentes de variáveis $X$. No pipeline desenvolvido, $n$ está definido como 30. A execução do pipeline pode ser configurada pelo arquivo de configuração no diretório `conf/base/parameters_optimization.yaml` descrito abaixo:

```yaml
opt_sim_params:
  gs_steps: 100
  bayesian_max_time: 4
  bayesian_max_it: 4000
  scenario_features:
    - ALIM_FLOT_CU_TOT
    - PH_ROUGHER_COND
    - P20_MOAGEM
  decision_variables:
    - VAZAO_AR_ROUGHER_COND
    - Espumante (g/t)_CD
    - ESPUMA_ROUGHER_COND
```

Neste arquivo, definem-se os seguintes parâmetros: 

* `scenario_features`: o conjunto de variáveis de contexto $X$. Estas variáveis serão utilizadas para encontar 30 cenários distintos na base de dados histórica;
* `decision_variables`: o conjunto de variáveis de decisão $V_d$;
* Parâmetros da otimização:
  * `gs_steps`: quantidade de valores testados para cada variável de decisão dentro do algoritmo grid-search. Deve-se levar em consideração que altos valores deste parâmetro irão resultar em tempos de execução demasiadamente longos. Deve-se considerar que o total de valores testados é $\text{gs-steps}^m$, sendo $m$ a quantidade de variáveis de decisão. Ou seja, para 3 variáveis de decisão com 100 gs_steps, tem-se 1 milhão de combinações. Para 5 variáveis de decisão, tem-se 10 bilhões de combinações. 200 milhões de combinações é um valor alto, porém com tempo de execução razoável por volta de 10 minutos (depende da capacidade de processamento da CPU utilizada) para cada otimização; 
  * `bayesian_max_time`: tempo máximo de execução da otimização bayesiana;
  * `bayesian_max_it`: quantidade de iterações máximas da otimização bayesiana.

Após a parametrização do arquivo de configuração, pode-se executar o pipeline com `kedro run --pipeline=optimization`. O tempo de execução depende dos parâmetros estabelecidos, mas a execução dura em média ~2 horas considerando 4 minutos por otimização bayesiana (`bayesian_max_it`), 100 passos no grid-search (`gs_steps`) e 3 variáveis de decisão.

A execução do pipeline resulta em um plot salvo na pasta `data/08_reporting/conc_cd_simulations_plot.png`, segue um exemplo abaixo:

<img src="Images/exemplo_otm.png" alt="Otimização" width="1000"/>

Neste plot, têm-se informações acerca da otimização para cada uma das variáveis de decisão e também de seus valores observados no histórico. As duas primeiras linhas se referem ao comportamento das variáveis de decisão em relação à função objetivo para cada cenário otimizado. Neste exemplo, a função objetivo a ser maximizada foi o valor predito pelo modelo de teor de cobre no CD. A escala dos gráficos vão de 0 a 1 que se traduzem no mínimo e máximo de teor de cobre encontrado para diferentes valores da variável de decisão em cada cenário.

A última linha de gráficos se refere um resumo dos valores ótimos para cada variável de decisão utilizando os otimizadores e também o observado no histórico.

Estas análises são úteis para verificar a diversidade das soluções sugeridas pelo otimizador para cada uma das variáveis de decisão, ou seja, nos distintos cenários o modelo escolhe um mesmo valor para uma determinada variável de decisão ou o modelo é capaz de levar o contexto em consideração para sugerir diferentes valores.



#### 3.8 <a id='section_38'></a>[Atualização do simulador MVP](#section_3)

A atualização do Simulador MVP é um processo essencial para garantir que o sistema esteja alinhado com as últimas informações em termos de dados, modelos preditivos e práticas operacionais. É recomendável realizar este processo de atualização com frequência, idealmente uma vez por semana, para assegurar a relevância e eficácia contínua do simulador. Nesta seção, detalhamos as etapas chave para manter o simulador atualizado, que incluindo:

- [Visão geral das pastas do WebApp](#secao_381)
- [Implementação de novos modelos preditivos](#secao_382)
- [Adição de novas features](#secao_383)
- [Atualização do Tipo de Otimizador](#secao_384)

#####  <a id='secao_381'></a> **Visão geral das pastas do WebApp**

`00_metadata`

-   Descrição: Armazena as variáveis e suas etapas no processo.
-   Arquivos principais:
    -   `etapas.yaml`: Dicionário contendo mapeamento das variáveis e suas etapas.

`01_raw_data`

-   Descrição: Contém os dados brutos inseridos pelos usuários.
-   Arquivos principais:
    -   `balanco_de_massas.xlsx`: Dados brutos do balanço de massas.
    -   `blend.xlsx`: Informações relacionadas ao blend.
    -   `carta_controle_pims.xlsx`: Dados de controle de processos PIMS.
    -   `laboratorio.xlsx` e `laboratorio_raiox.xlsx`: Dados brutos do laboratório e raio-X.
    -   `reagentes.xlsx`: Informações sobre os reagentes utilizados.

`02_intermediate`

-   Descrição: Armazena dados processados em um estágio intermediário.
-   Arquivos principais:
    -   `carta_controle_pims_pre.pq`: Dados pré-processados de controle PIMS.
    -   `balanco_de_massas_pre.pq`: Dados pré-processados do balanço de massas.
    -   `laboratorio_raiox_pre.pq` e `laboratorio_pre.pq`: Dados pré-processados do laboratório e raio-X.
    -   `reagentes_pre.pq`: Dados pré-processados dos reagentes.
    -   `blend_pre.pq`: Dados pré-processados de blend.

`03_primary`

-   Descrição: Contém dados primários processados e prontos para uso.
-   Arquivos principais:
    -   `merged_raw_data.pq`: Dados brutos mesclados.
    -   `filtered_data.pq`: Dados filtrados.
    -   `hourly_data.pq`: Dados processados em base horária.

`04_models`

-   Descrição: Armazena os modelos de machine learning.
-   Arquivos principais:
    -   `conformal_model_conc_cd_randomly.pickle`: Modelo conformal para concentração na CD.
    -   `ebm_conc_cd_randomly.pickle`: Modelo EBM para conetração na CD.

##### <a id='secao_382'></a> **Implementação de novos modelos preditivos**

1.   **Seleção do Modelo Preditivo:** A escolha do modelo mais adequado para integração no simulador é um passo crucial. Esta decisão deve ser tomada após a conclusão das fases de [otimização dos hiperparâmetros dos modelos preditivos](#section_34), [análise de erro/performance](#section_36) e a subsequente [análise da simulação do otimizador com o modelo preditivo](#section_37). Estes processos asseguram que o modelo selecionado esteja não apenas tecnicamente afinado com parâmetros e hiperparâmetros otimizados, mas também alinhado com as necessidades e realidades do negócio, especialmente em relação à variação das variáveis de decisão.

2.   **Implementação do Modelo Preditvo no Simulador:** Após a seleção do modelo preditivo, prossiga com a integração do modelo escolhido ao simulador. Este processo envolve copiar os modelos `ebm_conc_cd_randomly.pickle` e `conformal_model_conc_cd_randomly.pickle` encontrados em **data/06_models/** e colar no diretório **app/data/04_models** do simulador.

3. **Integração dos Dados:** Para a independência de importação de planilhas ou conexão com banco de dados no WebApp, é crucial integrar os dados de maneira adequada. Isso inclui copiar o arquivo `data/03_primary/merged_raw_data.pq` para o diretório `app/data/03_primary/`, renomeando-o para `hourly_data.pq`. Esse passo garante que o WebApp tenha acesso aos dados necessários.

##### <a id='secao_383'></a> **Adição de novas *features***

Em casos de novas *features* no modelo preditivo implementado, ou seja,  a adição de *features* ainda não presentes no dicionário `etapas.yaml`, será necessário adicionar o nome da etapa do processo de flotação referente (caso não exista), e o nome da variável juntamente com a sua unidade de medida. Esta adição garante que as novas *features* sejam corretamente integradas e reconhecidas no sistema dinâmico do webapp (o mesmo passo a passo funciona para as variáveis de decisão). 

<img src="Images/exemplo_adicao_variavel_webapp.png" alt="WebApp" width="1000"/>


####  <a id='secao_384'></a> **Atualização do Tipo de Otimizador**

Para garantir a eficácia das simulações, pode ser necessário alternar entre diferentes otimizadores seguindo as análises . Siga as instruções abaixo para atualizar o otimizador no WebApp:

1.  Configuração do Otimizador: Acesse `app\data\00_metadata\etapas.yaml` e localize a seção `otimizador`. Substitua o tipo atual pelo desejado (por exemplo, de "Bayesian" para "GridSearch").

2.  Aplicação das Mudanças: Salve as alterações em `etapas.yaml` e reinicie o WebApp para que a nova configuração tenha efeito.

<img src="Images/exemplo_modificar_otimizacao_webapp.png" alt="Otimizador" width="1000"/>

Para utilizar o MVP com os modelos atualizados, executar o comando `streamlit run app/Sobre.py` no terminal.


## 4 <a id='section_4'></a>[Fluxo de Uso](#toc0_) [&#8593;](#toc0_)


O fluxo de uso com todas as etapas a serem executadas está definido a seguir:

  - 4.1 [Processar Dados Novos](#section_41)
  - 4.2 [Calibrar Modelo Baseline e Executar Simulação de Otimização](#section_42)
  - 4.3 [Otimização de Hiperparâmetros do Modelo Campeão](#section_43)
  - 4.4 [Calibrar Modelo Campeão e Executar Simulação de Otimização](#section_44)
  - 4.5 [Comparar Resultados](#section_45)
  - 4.6 [Atualizar Modelo no MVP (WebApp)](#section_46)


#### 4.1  <a id='section_41'></a>[Processar Dados Novos](#section_4)

* Configurar o arquivo **conf/base/parameters_data_processing.yml** e executar o pipeline:

  ```bash
  kedro run --pipeline=data_processing
  ```

#### 4.2  <a id='section_42'></a>[Calibrar Modelo Baseline e Executar Simulação de Otimização](#section_4)

* Executar o notebook **Dynamic Feature.ipynb** e avaliar mudanças na dinâmica do processo.

* Configurar o arquivo **conf/base/parameters_generate_models_inputs.yml** e executar o pipeline:

  ```bash
  kedro run --pipeline=generate_models_inputs
  ```

* Configurar o arquivo **conf/base/parameters_data_science.yml** e executar o pipeline:
  
  ```bash
  kedro run --pipeline=data_science
  ```

* Configurar o arquivo **conf/base/parameters_optimization.yml** e executar o pipeline:

  ```bash
  kedro run --pipeline=optimization
  ```

#### 4.3  <a id='section_43'></a>[Otimização de Hiperparâmetros do Modelo Campeão](#section_4)

* Configurar o arquivo **conf/base/pparameters_models_optimization.yml** e executar o pipeline:

  ```bash
  kedro run --pipeline=models_optimization
  ```

#### 4.4  <a id='section_44'></a>[Calibrar Modelo Campeão e Executar Simulação de Otimização](#section_4)

* Executar o notebook **Dynamic Feature.ipynb** e avaliar mudanças na dinâmica do processo.

* Configurar o arquivo **conf/base/parameters_generate_models_inputs.yml** e executar o pipeline:

  ```bash
  kedro run --pipeline=generate_models_inputs
  ```

* Configurar o arquivo **conf/base/parameters_data_science.yml** e executar o pipeline:
  
  ```bash
  kedro run --pipeline=data_science
  ```

* Configurar o arquivo **conf/base/parameters_optimization.yml** e executar o pipeline:

  ```bash
  kedro run --pipeline=optimization
  ```

#### 4.5  <a id='section_45'></a>[Comparar Resultados](#section_4)

* Executar notebooks e avaliar as métricas de erro.

  * Modelagem-TeorCu-Cd-Baseline.ipynb
  * Modelagem-TeorCu-Cd new pipeline.ipynb
  * Modelagem-RM-Global-Baseline.ipynb
  * Modelagem-RM-Global new pipeline.ipynb

#### 4.6  <a id='section_46'></a>[Atualizar Modelo no MVP (WebApp)](#section_4)



1. Vá até o diretório **data/06_models**.

2. Localize os arquivos denominados `conformal_model_conc_cd_randomly.pickle` e `ebm_conc_cd_randomly.pickle`.

3. Copie ambos os arquivos.

4. Navegue até o diretório **app/data/04_models**.

5. Cole os arquivos previamente mencionados neste novo diretório.

6. (Opcional) Altere o arquivo **app/data/00_metadata/etapas.yml** caso o novo modelo possua *features* adicionais.

